In [1]:
import folium
import folium.plugins as plugins
from folium.plugins import HeatMap
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
#import geopandas
import random


covid_dataframe = pd.read_csv('spread_hist_by_state.csv')
covid_dataframe.columns

covid_data = covid_dataframe[['date', 'state', 'positive', 'negative','recovered', 'dataQualityGrade', 'lastUpdateEt']].loc[covid_dataframe['dataQualityGrade'] != 'F']
covid_data.head(3)

state_geo_df = pd.read_csv('state_latlong.csv')
covid_df = pd.merge(covid_data, state_geo_df, how='left', left_on="state", right_on='state')
#covid_df
#print(covid_df.loc[covid_df['latitude'] ==  ][:])


plot_df = covid_df[['latitude','longitude','date','positive', 'negative', 'state', 'recovered']].sort_values(by='date', axis=0, ascending=True).fillna(0)
print(plot_df.loc[plot_df['latitude'] == 0]['state'].unique())
plot_df

In [5]:
#gdf = geopandas.GeoDataFrame(
#    plot_df, geometry=geopandas.points_from_xy(plot_df.longitude, plot_df.latitude))
#gdf

In [6]:
#help(plugins.HeatMapWithTime)
#help(folium.Icon)

In [7]:
main_map = folium.Map(
    [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    tiles='stamentoner',
    zoom_start=4,
    max_opacity=0.9,
    blur=4
)

#heat_data_pos_grp = []
#heat_data_neg_grp = []
#heat_data_rec_grp = []
date_indx = []

layer_types = [{'title' : 'Covid : Tested Positive',
                'status' : 'positive',
                'show' : True,
                'plot_data' : [],
                'gr_val' : [.2,.9,.3],
                'gr_clr' : ['red','orange','red'],
                'rad' : 1.5
               },
               {'title' : 'Covid : Tested Negative',
                'status' : 'negative',
                'show' : False,
                'plot_data' : [],
                'gr_val' : [.9,.3,.3],
                'gr_clr' : ['green','yellow','lime'],
                'rad' : 1
               },
               {'title' : 'Covid : Recovered',
                'status' : 'recovered',
                'show' : False,
                'plot_data' : [],
                'gr_val' : [.5,.5,.5],
                'gr_clr' : ['blue','blue','blue'],
                'rad' : 1
               }]

for date_stp in plot_df['date'].unique():
    heat_df = plot_df.loc[plot_df['date'] == date_stp].dropna(axis=0, subset=['latitude', 'longitude', 'positive', 'negative', 'recovered', 'date'])
    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))    
    heat_data_pos_date = []
    heat_data_neg_date = []
    heat_data_rec_date = []
    for row in heat_df.itertuples():
        pos_magna = int(row.positive)
        neg_magna = int(row.negative)
        rec_magna = int(row.recovered)
        #if pos_magna > 1000: pos_magna = 1000
        #print(f'pos_magna [{pos_magna}]')
        if pos_magna > 0:
            #heat_data_pos_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.positive])
            #heat_point = [row.latitude, row.longitude, 1]
            heat_point = [row.latitude * random.uniform(1.0001,1.0005), row.longitude * random.uniform(1.0001,1.0005), 1]
            heat_data_pos_date.append(heat_point)
            #heat_point = [row.latitude, row.longitude, 1]
            #[heat_data_pos_date.append(heat_point) for i in range(pos_magna)]
        if neg_magna > 0:
            #heat_data_neg_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.negative])
            #heat_point = [row.latitude * random.uniform(.995,1.005), row.longitude * random.uniform(.995,1.005), 1]
            heat_point = [row.latitude, row.longitude, 1]
            heat_data_neg_date.append(heat_point) 
            #[heat_data_neg_date.append(heat_point) for i in range(neg_magna)]
        if rec_magna > 0:
            #heat_data_rec_date.append([row.latitude * random.uniform(1.0004,1.001), row.longitude * random.uniform(1.0004,1.001), row.recovered])
            #heat_point = [row.latitude * random.uniform(.995,1.005), row.longitude * random.uniform(.995,1.005), 1]
            heat_point = [row.latitude, row.longitude, 1]
            heat_data_rec_date.append(heat_point)
            #[heat_data_rec_date.append(heat_point) for i in range(rec_magna)]
            
    #print(f'heat_data_pos_date [{heat_data_pos_date}]')
    layer_types[0]['plot_data'].append(heat_data_pos_date)
    layer_types[1]['plot_data'].append(heat_data_neg_date)
    layer_types[2]['plot_data'].append(heat_data_rec_date)

print(f'layer_types[0]["plot_data"] [{len(layer_types[0]["plot_data"])}] date_indx [{len(date_indx)}]')
#print(f'layer_types[0][3] {"--"* 40}\n{layer_types[0][3]}')
#print(f'layer_types[0]["plot_data"] [{layer_types[0]["plot_data"]}]')

for layer_type in layer_types:

    #feature_sub_grp = folium.plugins.FeatureGroupSubGroup(map_Feature_Group,layer_type[0]+'_grp', show=layer_type[2])
    map_Feature_Group = folium.FeatureGroup(name=layer_type['title'], show=layer_type['show'])
    main_map.add_child(map_Feature_Group)
    #main_map.add_child(feature_sub_grp)

    Heat_mapw_time = plugins.HeatMapWithTime(
        data = layer_type['plot_data'],
        index = date_indx,
        auto_play = True,
        use_local_extrema = True,
        radius = layer_type['rad'],
        min_opacity = 0.5,
        scale_radius = True,
        name = layer_type['title'],
        gradient={layer_type['gr_val'][0]: layer_type['gr_clr'][0],
                  layer_type['gr_val'][1]: layer_type['gr_clr'][1],
                  layer_type['gr_val'][2]: layer_type['gr_clr'][2]}
    )
     #control=layer_type[2]

    main_map.add_child(Heat_mapw_time)

    
#marker_cluster = MarkerCluster().add_to(map_Feature_Group)
#folium.TileLayer(layer_type[0]).add_to(map)    
# add the layer control
folium.LayerControl().add_to(main_map)


main_map

layer_types[0]["plot_data"] [113] date_indx [113]


In [8]:
# extract lat, lon, and magnitude for the folium heatmap
from folium.plugins import HeatMap
df_copy = df[df.month>4].copy()
df_copy['count'] = 1
base_map = generateBaseMap()
HeatMap(data=df_copy[['pickup_latitude', 'pickup_longitude', 'count']].groupby(['pickup_latitude', 'pickup_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(base_map)

NameError: name 'df' is not defined

In [ ]:
    dt = str(date_stp)
    y = int(dt[:4])
    m = int(dt[4:6])
    d = int(dt[6:])
    date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))

In [ ]:
industriesDF = pd.read_csv('merged_industries.csv')
industriesDF

In [9]:
#data=sliceDF[[‘Lat’, ‘Lng’, ‘count’]].groupby([‘Lat’, ‘Lng’]).sum().reset_index().values.tolist()


main_map = folium.Map(
    [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    tiles='stamentoner',
    zoom_start=4,
    max_opacity=0.9,
    blur=4
)

#heat_data_pos_grp = []
#heat_data_neg_grp = []
#heat_data_rec_grp = []
date_indx = []

layer_types = [{'title' : 'Covid : Tested Positive',
                'status' : 'positive',
                'show' : True,
                'plot_data' : [],
                'gr_val' : [.2,.9,.3],
                'gr_clr' : ['red','orange','red'],
                'rad' : 1.5
               },
               {'title' : 'Covid : Tested Negative',
                'status' : 'negative',
                'show' : False,
                'plot_data' : [],
                'gr_val' : [.9,.3,.3],
                'gr_clr' : ['green','yellow','lime'],
                'rad' : 1
               },
               {'title' : 'Covid : Recovered',
                'status' : 'recovered',
                'show' : False,
                'plot_data' : [],
                'gr_val' : [.5,.5,.5],
                'gr_clr' : ['blue','blue','blue'],
                'rad' : 1
               }]

heat_df = plot_df.dropna(axis=0, subset=['latitude', 'longitude', 'positive', 'negative', 'recovered', 'date'])

heat_df['positive'] = heat_df['positive'].astype(int)
heat_df['negative'] = heat_df['negative'].astype(int)
heat_df['recovered'] = heat_df['recovered'].astype(int)#.map('${:,.2f}'.format)
#dt = str(date_stp)
#y = int(dt[:4])
#m = int(dt[4:6])
#d = int(dt[6:])
#date_indx.append(datetime(y,m,d).strftime("%Y-%m-%d"))    
try:
    for layer_type in layer_types:

        Heat_mapw_time = plugins.HeatMapWithTime(
            data = heat_df[['latitude', 'longitude', layer_type['status']]].groupby(['latitude', 'longitude']).sum().reset_index().values.tolist(),
            auto_play = True,
            use_local_extrema = True,
            radius = layer_type['rad'],
            min_opacity = 0.5,
            scale_radius = True,
            name = layer_type['title'],
            gradient={layer_type['gr_val'][0]: layer_type['gr_clr'][0],
                      layer_type['gr_val'][1]: layer_type['gr_clr'][1],
                      layer_type['gr_val'][2]: layer_type['gr_clr'][2]}
        )
         #control=layer_type[2]
        #feature_sub_grp = folium.plugins.FeatureGroupSubGroup(map_Feature_Group,layer_type[0]+'_grp', show=layer_type[2])
        map_Feature_Group = folium.FeatureGroup(name=layer_type['title'], show=layer_type['show'])
        main_map.add_child(map_Feature_Group)
        #main_map.add_child(feature_sub_grp)
        main_map.add_child(Heat_mapw_time)
        #print(f'\n{layer_type["status"]} data\n{"--"*40}\n{Heat_mapw_time.data}')

    #marker_cluster = MarkerCluster().add_to(map_Feature_Group)
    #folium.TileLayer(layer_type[0]).add_to(map)    
    # add the layer control
    folium.LayerControl().add_to(main_map)

except Exception as e:
    print(f'An ERROR occured [{e}]')
main_map

In [ ]:
'''


#g1 = folium.plugins.FeatureGroupSubGroup(mcg, 'g1') # First group, in mcg
#g2 = folium.plugins.FeatureGroupSubGroup(mcg, 'g2') # Second group, in mcg
#m.add_child(mcg)
#m.add_child(g1)
#m.add_child(g2)
#g1.add_child(folium.Marker([0,0]))
#g2.add_child(folium.Marker([0,1]))
#folium.LayerControl().add_to(m)



#  zoom_start=6
params = {
    'location':[plot_df['latitude'].mean(),plot_df['longitude'].mean()],
    'tiles':'stamentoner',
    'min_lat':plot_df['latitude'].min(),
    'max_lat':plot_df['latitude'].max(),
    'min_lon':plot_df['longitude'].min(),
    'max_lon':plot_df['longitude'].max()
}
#        min_lat=plot_df['latitude'].min(),
#        max_lat=plot_df['latitude'].max(),
#        min_lon=plot_df['longitude'].min(),
#        max_lon=plot_df['longitude'].max(),
covid_map = folium.Map(
        [plot_df['latitude'].mean(),plot_df['longitude'].mean()],
        tiles='stamentoner',
        zoom_start=5
    )


#heat_df = plot_df.dropna(axis=0, subset=['latitude','longitude'])
#heat_data = [[row['latitude'],row['longitude']] for index, row in heat_df.iterrows()]

heat_df = plot_df.loc[plot_df['date'] == 20200315].dropna(axis=0, subset=['latitude','longitude', 'total'])
print(f'heat_df {len(heat_df)}')
#heat_df['weight'] = heat_df['total'].astype(float)
#heat_data = [[[row['latitude'],row['longitude']] for index, row in heat_df[heat_df['weight'] == i].iterrows()] for i in range(0,plot_df['total'].max())]

#heat_time = list(heat_df.dropna(axis=0, subset=['date']))
heat_time =  [[row['date']] for index, row in heat_df.iterrows()]
#heat_time =  [[index] for index, row in heat_df.iterrows()]

#heat_time = plot_df[])#.toList()
heat_data = []
for row in heat_df.itertuples():
    #print(row.total)
    heat_data.append([row.latitude,row.longitude, row.total])# for i in range(0, row.total)]
#    [heat_data.append([row.latitude,row.longitude, row.total]) for i in range(0, row.total)]
#heat_data = [[row['latitude'],row['longitude'],row['total']] for index, row in heat_df.iterrows()]
len(heat_data)

                
#     for index, row in heat_df.iterrows()]
print(heat_data[:3])

HeatMap(heat_data).add_to(covid_map)
covid_map'''